In [1]:
import numpy as np

In [2]:
class Tensor:
    def __init__(self,h=1,w=1,weight=None,trainable=False,grad=None):
        self.weight = weight
        self.trainable=trainable
        self.grad=grad
    
    # not sure if we need child member
    def __call__(self,child):
        self.child = child


In [3]:
class Act(Tensor):
    def __init__(self):
        super().__init__()
    
    def forward(self,x):
        raise NotImplementedError
    
    def backward(self,bpass):
        if self.child is not None:
            self.child.backward(np.multiply(self.grad,bpass))


In [4]:
class relu(Act):
    def __init__(self):
        super().__init__()
    
    def forward(self,x):
        pass